we first ask the model from transformer

In [6]:
from transformers import pipeline, set_seed
generator=pipeline("text-generation",model='gpt2')
set_seed(42)
generator("Hello, I am greg", max_length=30, num_return_sequences=5)

Device set to use mps:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=30) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[{'generated_text': "Hello, I am gregarious and am incredibly lucky to have a great relationship with my best friend, and I am excited to discuss how I came to love him so much. His life was a whirlwind for me. I had to learn to live with my gut, my emotions, and my emotions.\n\nI started to love my friend with open eyes and a lot of hard work, but I also had to keep my mind open and focused on my own life. I am so grateful to have him with me. I will always love him. When I was a kid I was so embarrassed to have so many people around me, but now that I am in my 20s and I am seeing more and more people, I'm more open and open.\n\nAt the same time, I like him. I really want him to get through his transition to become an adult. I'm thrilled to have him. I will always love him. When I was a kid, I was so excited to have a big, happy family, but now, I'm sad to see my friends disappear like these, and I want them to get through their transition. I've been talking to a lot of people about t

In [ ]:
from model import GPT
import torch
import torch.nn.functional as F
import tiktoken

device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

model=GPT.from_pretrained('gpt2').to(device) 
sequence_max=30
n_sequence=5
enc=tiktoken.get_encoding("gpt2")
tokens=enc.encode("Hello, I'm a language model,")
tokens=torch.tensor(tokens,dtype=torch.long)
tokens=tokens.unsqueeze(0).repeat(n_sequence,1)
x=tokens.to(device)


torch.manual_seed(42)
torch.cuda.manual_seed(42)
while x.size(1)< sequence_max:
    with torch.no_grad():
        logits,_=model(x)
        logits=logits[:,-1,:]
        prob=F.softmax(logits,dim=-1)
        topk_probs,topk_indices=torch.topk(prob,50, dim=-1)

        ix=torch.multinomial(topk_probs,1)

        xcol=torch.gather(topk_indices,-1, ix)

        x=torch.cat((x, xcol),dim=1)

for i in range(n_sequence):
    token=x[i,:sequence_max].tolist()
    decoded=enc.decode(token)
    print(">",decoded)


loading weights from pretrained gpt: gpt2
> Hello, I'm a language model, not a science. I'm a language designer. I want to write, I want to think. I want
> Hello, I'm a language model, I use an English sentence structure, I like words over sentences.

"That's OK I'll look
> Hello, I'm a language model, not just another language." This isn't a "language model?" It's an idea. So far, what
> Hello, I'm a language model, not a programming model. I'm not a theoretical computer model - you read that right - because my ideas are
> Hello, I'm a language model, I teach myself.

I want to know more about how languages work and why they could be used.


In [20]:
from transformers import GPT2LMHeadModel

model2=GPT2LMHeadModel.from_pretrained("gpt2").to(device)
model2.eval()
sequence_max=30
n_sequence=5
enc=tiktoken.get_encoding("gpt2")
tokens=enc.encode("Hello, I'm a language model,")
tokens=torch.tensor(tokens,dtype=torch.long)
tokens=tokens.unsqueeze(0).repeat(n_sequence,1)
x=tokens.to(device)


torch.manual_seed(42)
torch.cuda.manual_seed(42)
while x.size(1)< sequence_max:
    with torch.no_grad():
        logits=model2(x)[0]
        logits=logits[:,-1,:]
        prob=F.softmax(logits,dim=-1)
        topk_probs,topk_indices=torch.topk(prob,50, dim=-1)

        ix=torch.multinomial(topk_probs,1)

        xcol=torch.gather(topk_indices,-1, ix)

        x=torch.cat((x, xcol),dim=1)

for i in range(n_sequence):
    token=x[i,:sequence_max].tolist()
    decoded=enc.decode(token)
    print(">",decoded)

> Hello, I'm a language model, not a science. I'm a language designer. I want to write, I want to think. I want
> Hello, I'm a language model, I use an English sentence structure, I like words over sentences.

"That's OK I'll look
> Hello, I'm a language model, not just another language." This isn't a "language model?" It's an idea. So far, what
> Hello, I'm a language model, not a programming model. I'm not a theoretical computer model - you read that right - because my ideas are
> Hello, I'm a language model, I teach myself.

I want to know more about how languages work and why they could be used.
